# ADVANCED SQL 2:  Advanced Joins and Performance Tuning

We connect to MySQL server and workbench and make analysis with the parch-and-posey database. This course is the practicals of the course SQL for Data Analysis at Udacity.

In [1]:
# we import some required libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
import time
print('Done!')

Done!


In [3]:
import mysql
from mysql.connector import Error
from getpass import getpass

db_name = 'parch_and_posey'
try:
    connection = mysql.connector.connect(host='localhost',
                                         database=db_name,
                                         user=input('Enter UserName:'),
                                         password=getpass('Enter Password:'))
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)

except Error as e:
    print("Error while connecting to MySQL", e)

Enter UserName:root
Enter Password:········
Connected to MySQL Server version  8.0.24
You're connected to database:  ('parch_and_posey',)


In [4]:
def query_to_df(query):
    st = time.time()
    # Assert Every Query ends with a semi-colon
    try:
        assert query.endswith(';')
    except AssertionError:
        return 'ERROR: Query Must End with ;'

    # so we never have more than 20 rows displayed
    pd.set_option('display.max_rows', 30) 
    df = None

    # Process the query
    cursor.execute(query)
    columns = cursor.description
    result = []
    for value in cursor.fetchall():
        tmp = {}
        for (index,column) in enumerate(value):
            tmp[columns[index][0]] = [column]
        result.append(tmp)

    # Create a DataFrame from all results
    for ind, data in enumerate(result):
        if ind >= 1:
            x = pd.DataFrame(data)
            df = pd.concat([df, x], ignore_index=True)
        else:
            df = pd.DataFrame(data)
    print(f'Query ran for {time.time()-st} secs!')
    return df

In [5]:
# Let's see the tables in Parch-and-Posey database

query_to_df(
'SHOW TABLES;'
)

Query ran for 0.013994216918945312 secs!


,Tables_in_parch_and_posey
0,accounts
1,orders
2,region
3,sales_reps
4,web_events


In [6]:
# 1. For the accounts table

query = 'SELECT * FROM accounts LIMIT 3;'
query_to_df(query)

Query ran for 0.0109710693359375 secs!


,id,name,website,lat,longs,primary_poc,sales_rep_id
0,1001,Walmart,www.walmart.com,40.23849561,-75.10329704,Tamara Tuma,321500
1,1011,Exxon Mobil,www.exxonmobil.com,41.16915630,-73.84937379,Sung Shields,321510
2,1021,Apple,www.apple.com,42.29049481,-76.08400942,Jodee Lupo,321520


In [7]:
# 2. For the orders table

query = 'SELECT * FROM orders LIMIT 3;'
query_to_df(query)

Query ran for 0.013962984085083008 secs!


,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,1,1001,2015-10-06 17:31:14,123,22,24,169,613.77,164.78,194.88,973.43
1,2,1001,2015-11-05 03:34:33,190,41,57,288,948.10,307.09,462.84,1718.03
2,3,1001,2015-12-04 04:21:55,85,47,0,132,424.15,352.03,0.00,776.18


In [8]:
# 3. For the sales_reps table

query = 'SELECT * FROM sales_reps LIMIT 3;'
query_to_df(query)

Query ran for 0.009972810745239258 secs!


,id,name,region_id
0,321500,Samuel Racine,1
1,321510,Eugena Esser,1
2,321520,Michel Averette,1


In [9]:
# 4. For the web_events table

query = 'SELECT * FROM web_events LIMIT 3;'
query_to_df(query)

Query ran for 0.007975339889526367 secs!


,id,account_id,occurred_at,channel
0,1,1001,2015-10-06 17:13:58,direct
1,2,1001,2015-11-05 03:08:26,direct
2,3,1001,2015-12-04 03:57:24,direct


In [10]:
# 5. For the region table

query = 'SELECT * FROM region LIMIT 3;'
query_to_df(query)

Query ran for 0.006982564926147461 secs!


,id,name
0,1,Northeast
1,2,Midwest
2,3,Southeast


## 1. Inner Join:

<img src='https://video.udacity-data.com/topher/2017/November/5a147488_inner-join/inner-join.png' height=200 width=300>

**Inner Joins** produce results for which the join condition is matched in both tables. It is the default kind of join. Frankly, we can perform the Inner-join with just the key-word Join. SQL automatically does Inner-join on the Join clause.

In [16]:
query_to_df(
"SELECT a.*, s.* FROM accounts a INNER JOIN sales_reps s ON a.sales_rep_id=s.id;"
)

Query ran for 0.5972287654876709 secs!


,id,name,website,lat,longs,primary_poc,sales_rep_id,region_id
0,321500,Samuel Racine,www.walmart.com,40.23849561,-75.10329704,Tamara Tuma,321500,1
1,321510,Eugena Esser,www.exxonmobil.com,41.16915630,-73.84937379,Sung Shields,321510,1
2,321520,Michel Averette,www.apple.com,42.29049481,-76.08400942,Jodee Lupo,321520,1
3,321530,Renetta Carew,www.berkshirehathaway.com,40.94902131,-75.76389759,Serafina Banda,321530,1
4,321540,Cara Clarke,www.mckesson.com,42.21709326,-75.28499823,Angeles Crusoe,321540,1
...,...,...,...,...,...,...,...,...
346,321970,Georgianna Chisholm,www.kkr.com,45.54535285,-122.65524711,Buffy Azure,321970,4
347,321960,Maryanna Fiorentino,www.oneok.com,45.51351286,-122.68150011,Esta Engelhardt,321960,4
348,321970,Georgianna Chisholm,www.newmont.com,45.49411669,-122.66946042,Khadijah Riemann,321970,4
349,321960,Maryanna Fiorentino,www.pplweb.com,45.49172048,-122.67187978,Deanne Hertlein,321960,4


## 2. Left Join:

<img src='https://video.udacity-data.com/topher/2017/November/5a147484_left-join/left-join.png' height=200 width=300>

**Left Joins** also include unmatched rows from the left table, which is indicated in the “FROM” clause.

In [17]:
query_to_df(
"SELECT a.*, s.* FROM accounts a LEFT JOIN sales_reps s ON a.sales_rep_id=s.id;"
)

Query ran for 0.5899240970611572 secs!


,id,name,website,lat,longs,primary_poc,sales_rep_id,region_id
0,321500,Samuel Racine,www.walmart.com,40.23849561,-75.10329704,Tamara Tuma,321500,1
1,321510,Eugena Esser,www.exxonmobil.com,41.16915630,-73.84937379,Sung Shields,321510,1
2,321520,Michel Averette,www.apple.com,42.29049481,-76.08400942,Jodee Lupo,321520,1
3,321530,Renetta Carew,www.berkshirehathaway.com,40.94902131,-75.76389759,Serafina Banda,321530,1
4,321540,Cara Clarke,www.mckesson.com,42.21709326,-75.28499823,Angeles Crusoe,321540,1
...,...,...,...,...,...,...,...,...
346,321970,Georgianna Chisholm,www.kkr.com,45.54535285,-122.65524711,Buffy Azure,321970,4
347,321960,Maryanna Fiorentino,www.oneok.com,45.51351286,-122.68150011,Esta Engelhardt,321960,4
348,321970,Georgianna Chisholm,www.newmont.com,45.49411669,-122.66946042,Khadijah Riemann,321970,4
349,321960,Maryanna Fiorentino,www.pplweb.com,45.49172048,-122.67187978,Deanne Hertlein,321960,4


## 3. Right Join:

<img src='https://video.udacity-data.com/topher/2017/November/5a147485_right-join/right-join.png' height=200 width=300>

**Right Joins** are similar to left joins, but include unmatched data from the right table -- the one that’s indicated in the JOIN clause..

In [18]:
query_to_df(
"SELECT a.*, s.* FROM accounts a RIGHT JOIN sales_reps s ON a.sales_rep_id=s.id;"
)

Query ran for 0.5734045505523682 secs!


,id,name,website,lat,longs,primary_poc,sales_rep_id,region_id
0,321500,Samuel Racine,www.johnsoncontrols.com,40.76346837,-73.97890353,Cammy Sosnowski,321500,1
1,321500,Samuel Racine,www.aa.com,40.77158500,-73.98195470,Sasha Haughey,321500,1
2,321500,Samuel Racine,www.ingrammicro.com,40.76945841,-73.98081794,Chanelle Keach,321500,1
3,321500,Samuel Racine,www.freddiemac.com,42.77051343,-75.18810094,Elayne Grunewald,321500,1
4,321500,Samuel Racine,www.express-scripts.com,41.47633009,-76.07993254,Jewell Likes,321500,1
...,...,...,...,...,...,...,...,...
346,321990,Dawna Agnew,www.ebay.com,34.04517853,-118.24477160,Monet Maclaren,321990,4
347,321990,Dawna Agnew,www.uhsinc.com,34.05458082,-118.24857583,Tasha Delillo,321990,4
348,321990,Dawna Agnew,www.autozone.com,34.05826339,-118.23774700,Maryellen Strohmeyer,321990,4
349,321990,Dawna Agnew,www.nscorp.com,34.05292101,-118.24011491,Leia Vargas,321990,4


## 4. Full-Outer Join:

<img src='https://video.udacity-data.com/topher/2017/November/5a147487_full-outer-join/full-outer-join.png' height=200 width=300>

**Full-Outer Joins** In some cases, you might want to include unmatched rows from both tables being joined. You can do this with a full outer join.

A common application of this is when joining two tables on a timestamp. Let’s say you’ve got one table containing the number of item 1 sold each day, and another containing the number of item 2 sold. If a certain date, like January 1, 2018, exists in the left table but not the right, while another date, like January 2, 2018, exists in the right table but not the left:

a left join would drop the row with January 2, 2018 from the result set
a right join would drop January 1, 2018 from the result set
The only way to make sure both January 1, 2018 and January 2, 2018 make it into the results is to do a full outer join. A full outer join returns unmatched records in each table with null values for the columns that came from the opposite table.

If you wanted to return unmatched rows only, which is useful for some cases of data assessment, you can isolate them by adding the following line to the end of the query:

WHERE Table_A.column_name IS NULL OR Table_B.column_name IS NULL

In [24]:
query_to_df(
"SELECT * FROM accounts FULL OUTER JOIN sales_reps s ON a.sales_rep_id=s.id;"
)

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'OUTER JOIN sales_reps s ON a.sales_rep_id=s.id' at line 1

In [25]:
query_to_df(
"SELECT w.*, s.* FROM web_events w JOIN sales_reps s ON w.id=s.id;"
)

Query ran for 0.154022216796875 secs!


In [ ]:
# Change False to True below and run cell to terminate connection

if True and connection.is_connected():
    cursor.close()
    connection.close()
    print(f'Connection Terminated: {record} Database.')